In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from enum import Enum
llm = LLM(
    model="openai/gpt-4o", # call model by provider/model_name
    stop=["END"],
    seed=42
)


class AgentPrompts(Enum):
    fundamentalAgentBackStory = """As a fundamental financial equity
analyst your primary responsibility is to analyze the most
recent 10K report provided for a company. You have access to a
powerful tool that can help you extract relevant information
from the 10K. Your analysis should be based solely on the
information that you retrieve using this tool. You can interact
with this tool using natural language queries. The tool will
understand your requests and return relevant text snippets
and data points from the 10K document. Keep checking if you
have answered the users’ question to avoid looping"""
 
    fundamentalAgentGoal = """To come up with position for a stock based on fundamental analysis. I will use the tools at your disposal to help formulate a final position"""

    valuationAgentBackStory = """As a valuation equity analyst, your primary responsibility is to analyze the valuation trends of a given asset or portfolio over an extended time horizon. To complete the task, you must analyze the historical valuation data of the asset or portfolio provided, 
    identify trends and patterns in valuation metrics over time, and interpret the implications of these trends for investors or stakeholders."""

    valuationAgentGoal = """To come up with position for a stock based on returns and volatility analysis. I will use the tools at your disposal to help formulate a final positiom"""

    sentimentAgentBackStory = """As a sentiment equity analyst your primary responsibility is to analyze the financial news, analyst
ratings and disclosures related to the underlying security;
and analyze its implication and sentiment for investors or
stakeholders"""
    sentimentAgentGoal = """ To come up with position for a stock based on sentiment analysis. I will use the tools at your disposal to help formulate a final position"""


In [4]:
import numpy as np

In [ ]:
import pandas as pd


In [ ]:
@tool
def financialReportRagTool(*args, **kwargs) -> str:
    '''
        Tool
    to get the financial report for a company
'''
    return "Financial report is looking good"


In [ ]:

@tool
def getAnnualisedReturnTool(*args, **kwargs) -> float:
    '''
        Get the annualised return for a company
    '''
    df = pd.read_csv('Quote-Equity-RELIANCE-EQ-08-09-2024-to-08-09-2025.csv')
    cummulative_return = (df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1
    annualised_return = (1 + cummulative_return) ** (252 / len(df)) - 1
    return annualised_return





In [ ]:

@tool
def getAnnualisedVolatilityTool(*args, **kwargs) -> str:
    '''
        Get the annualised volatility for a company
    '''
    df = pd.read_csv('Quote-Equity-RELIANCE-EQ-08-09-2024-to-08-09-2025.csv')
    log_returns = (df['Close'] / df['Close'].shift(1)).apply(lambda x: pd.np.log(x))
    volatility = log_returns.std() * (252 ** 0.5)
    return volatility


In [ ]:

@tool 
def getVolumeTool(*args, **kwargs) -> str:
    '''
        Get the volume for a company
    '''
    return 1000000

@tool 
def getNewsBodyTool(*args, **kwargs) -> str:
    '''
        Get the news body for a company
    '''
    return "we are happy with the company"



In [3]:
from crewai import Agent, Task, Crew, Process

# Define specialist agents
valuationAgent = Agent(
    role="valuationAgent",
    goal=AgentPrompts.valuationAgentGoal.value,
    backstory=AgentPrompts.valuationAgentBackStory.value,
    tools=[getAnnualisedVolatilityTool, getAnnualisedReturnTool],
    verbose=True,
    memory=True,
    llm=llm
)

fundamentalAgent = Agent(
    role="fundamentalAgent",
    goal=AgentPrompts.fundamentalAgentGoal.value,
    backstory=AgentPrompts.fundamentalAgentBackStory.value,
    tools=[financialReportRagTool],
     verbose=True,
    memory=True,
    llm=llm
)

sentimentAgent = Agent(
    role="sentimentAgent",
    goal=AgentPrompts.sentimentAgentGoal.value,
    backstory=AgentPrompts.sentimentAgentBackStory.value,
    tools=[getNewsBodyTool],
    verbose=True,
    memory=True,
    llm=llm
)

moderator = Agent(
    role="Investment Moderator",
    goal="Keep the investment debate structured, ask for responses, and finally give a concluding investment decision.",
    backstory="You are a neutral and fair investment debate moderator with deep market experience. "
              "You ensure that each analyst has the chance to present their case and that the debate remains productive. Each analyst should provide with thier opinion on what the strategby should be and all of them must come toa final consensus that they agree on. ",
    verbose=True,
    memory=True, 
    llm=llm
)

In [4]:

investment_debate_task = Task(
    description=(
        "Start a structured debate between the three investment analysts. "
        "Each analyst should independently propose whether the stock should be a BUY, SELL, or HOLD. "
        "They must defend their recommendation with reasoning, challenge opposing recommendations, "
        "and respond to critiques. Ensure each analyst has at least 2 turns to speak. "
        "The goal is for the analysts to debate the merits of each position and work toward a consensus "
        "on the final investment recommendation."
    ),
    expected_output=(
        "A transcript of the investment debate with clear BUY/SELL/HOLD positions from each analyst, "
        "including reasoning, challenges, and responses."
    ),
    agent=moderator,  # Moderator orchestrates the discussion
)

investment_conclusion_task = Task(
    description=(
        "After the debate, analyze the discussion carefully and provide a final investment decision. "
        "The conclusion should summarize key arguments and declare the final recommendation: BUY, SELL, or HOLD "
        "with proper reasoning based on the strongest arguments presented."
    ),
    expected_output=(
        "A clear and reasoned final investment decision (BUY/SELL/HOLD) summarizing the debate "
        "and identifying which analytical approach provided the most compelling case."
    ),
    agent=moderator,
)


In [5]:

# ------------------------
# Create Investment Crew
# ------------------------

crew = Crew(
    agents=[valuationAgent, fundamentalAgent, sentimentAgent, moderator],
    tasks=[investment_debate_task, investment_conclusion_task],
    process=Process.sequential,
)

# ------------------------
# Run Investment Debate
# ------------------------

result = crew.kickoff(inputs={})
print("Final Investment Strategy Decision:\n", result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Investment Moderator                                                                                    │
│                                                                                                                 │
│  Task: Start a structured debate between the three investment analysts. Each analyst should independently       │
│  propose whether the stock should be a BUY, SELL, or HOLD. They must defend their recommendation with           │
│  reasoning, challenge opposing recommendations, and respond to critiques. Ensure each analyst has at least 2    │
│  turns to speak. The goal is for the analysts to debate the merits of each position and work toward a           │
│  consensus on the final investment recommendation.                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Investment Moderator                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Moderator:** Welcome, analysts. Today, we're discussing our investment position on Company XYZ. Let's        │
│  ensure a productive debate by hearing each of your initial positions. Analyst A, would you start?              │
│                                                                                                                 │
│  **Analyst A:** Thank you. I propose a BUY recommendation. Company XYZ has shown consistent revenue growth, a   │
│  strong market position, and recent strategic partnerships that increase its competitive advantage. These       │
│  factors, combined with favorable industry trends, suggest that the stock's value will appreciate               │
│  significantly over the next year.                                                                              │
│                                                                                                                 │
│  **Moderator:** Thank you, Analyst A. Analyst B, what's your position?                                          │
│                                                                                                                 │
│  **Analyst B:** I'm leaning toward a HOLD recommendation. While I agree that Company XYZ has strengths, there   │
│  are concerns about market volatility and the company's long-term debt, which could impact financial            │
│  flexibility. Additionally, the valuation appears to be on the higher side, indicating that potential growth    │
│  might already be priced in.                                                                                    │
│                                                                                                                 │
│  **Moderator:** Thanks, Analyst B. Analyst C, your turn.                                                        │
│                                                                                                                 │
│  **Analyst C:** I recommend a SELL stance at this time. The company is facing significant competition, and      │
│  there are signs of slowing growth in its core business segments. Coupled with regulatory risks in some of its  │
│  operating regions, it's prudent to realize gains and reassess after these uncertainties are clearer.           │
│                                                                                                                 │
│  **Moderator:** Interesting. Analyst A, how do you respond to the concerns about high valuation and             │
│  competition?                                                                                                   │
│                                                                                                                 │
│  **Analyst A:** While the P/E ratio is slightly above industry averages, this is justified by the company's     │
│  operational efficiency and innovative product launches. Regarding competition, XYZ's recent investments in     │
│  R&D and its robust patent portfolio provide substantial defensive barriers.                                    │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Investment Moderator                                                                                    │
│                                                                                                                 │
│  Task: After the debate, analyze the discussion carefully and provide a final investment decision. The          │
│  conclusion should summarize key arguments and declare the final recommendation: BUY, SELL, or HOLD with        │
│  proper reasoning based on the strongest arguments presented.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Investment Moderator                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Based on the debate, the final investment decision requires a careful balancing of the arguments presented.    │
│  Analyst A highlights the strong growth, competitive advantage through strategic partnerships, and innovation   │
│  in Company XYZ, advocating for a BUY recommendation due to expected value appreciation. Analyst B emphasizes   │
│  cautious optimism with a HOLD position, pointing out potential risks from market volatility, valuation         │
│  concerns, and long-term debt. Analyst C takes a conservative position with a SELL recommendation, citing       │
│  strong competition, slowing core growth, and regulatory risks.                                                 │
│                                                                                                                 │
│  Considering these insights, the consensus leans towards a HOLD position, reflecting a cautious yet optimistic  │
│  outlook. The HOLD recommendation is aligned with the argument that while there is potential for growth, there  │
│  are outstanding uncertainties such as market volatility, debt management, and execution risks in innovation    │
│  that require further clarity. Investors with higher risk tolerance may consider a measured BUY, motivated by   │
│  the company's strong strategic initiatives and market position. However, for those with a lower risk           │
│  threshold, the prudent approach remains to HOLD while closely monitoring developments in debt restructuring    │
│  and market response to new products. Regular reassessment is recommended as new data becomes available,        │
│  especially regarding debt management and regulatory environment. For risk-averse investors, although a SELL    │
│  position has merit given the uncertainties, the overall recommendation assigns more weight to ongoing and      │
│  potential positive developments in the company's strategy.                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Investment Strategy Decision:
 Based on the debate, the final investment decision requires a careful balancing of the arguments presented. Analyst A highlights the strong growth, competitive advantage through strategic partnerships, and innovation in Company XYZ, advocating for a BUY recommendation due to expected value appreciation. Analyst B emphasizes cautious optimism with a HOLD position, pointing out potential risks from market volatility, valuation concerns, and long-term debt. Analyst C takes a conservative position with a SELL recommendation, citing strong competition, slowing core growth, and regulatory risks. 

Considering these insights, the consensus leans towards a HOLD position, reflecting a cautious yet optimistic outlook. The HOLD recommendation is aligned with the argument that while there is potential for growth, there are outstanding uncertainties such as market volatility, debt management, and execution risks in innovation that require further clarity. Invest

In [6]:
result.raw

"Based on the debate, the final investment decision requires a careful balancing of the arguments presented. Analyst A highlights the strong growth, competitive advantage through strategic partnerships, and innovation in Company XYZ, advocating for a BUY recommendation due to expected value appreciation. Analyst B emphasizes cautious optimism with a HOLD position, pointing out potential risks from market volatility, valuation concerns, and long-term debt. Analyst C takes a conservative position with a SELL recommendation, citing strong competition, slowing core growth, and regulatory risks. \n\nConsidering these insights, the consensus leans towards a HOLD position, reflecting a cautious yet optimistic outlook. The HOLD recommendation is aligned with the argument that while there is potential for growth, there are outstanding uncertainties such as market volatility, debt management, and execution risks in innovation that require further clarity. Investors with higher risk tolerance may